# Build evaluation data

The following notebook takes a random sample of 200 rows from the evaluation data in the project, then uses the Surgeo library to add the necessary metadata. Finally, it saves the result as a CSV file in the project, so you can download it and use it for OpenScale evaluations. You can run all cells in the notebook at once. Each run will generate a different evaluation file named `policy_risk_openscale_eval.csv`.

Read in the evaluation data.

In [ ]:
from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space()

import pandas as pd

df_data_1 = pd.read_csv(wslib.mount.get_data_path('policy_risk_evaluation.csv'))
df_data_1.head()

Import the `os` library and identify the project.

In [ ]:
import os

os.environ['PROJECT_ID']

Randomly sample 200 rows of the evaluation data.

In [ ]:
evaluation_df = df_data_1.sample(n=200)
evaluation_df.head()

Ensure that the Surgeo library is installed to generate the `MINORITY` metadata column. For more info on the Surgeo library, see the `Model orchestration` notebook in this project.

In [ ]:
try:
    import surgeo
except:
    !pip install -U surgeo
    import surgeo

Create a new instance of the Surgeo model.

In [ ]:
sg = surgeo.SurgeoModel()

Get the necessary information for the Surgeo model as pandas series.

In [ ]:
names = pd.Series(evaluation_df['LAST_NAME'].values)
zips = pd.Series(evaluation_df['ZIPCODE'].values)

Use the surgeo model to get ethnicity probabilities for the 200 rows of the sample evaluation set.

In [ ]:
sg_results = sg.get_probabilities(names, zips)
sg_results

Define a function to determine if the policy holder is the member of a minority group. For simplicity's sake, assume that if the probability that they are white (Caucasian) is less than 50%, then they are a minority. Note that this is US-specific, as the training data is for a US city.

In [ ]:
def assign_value(percent):
    if percent > 0.5:
        return 'NON-MINORITY'
    return 'MINORITY'

Apply the defined function to each row of the results.

In [ ]:
sg_results["MINORITY"] = sg_results.apply(lambda row: assign_value(row["white"]), axis=1)
sg_results.head()

Add the results of the Surgeo model to the evaluation sample dataframe. Drop the `ZIPCODE` column, since it will no longer be used. Re-order the columns so that the label column (`RISK`) is at the end, which can help reduce errors when loading the evaluation data into OpenScale.

In [ ]:
evaluation_df['MINORITY'] = sg_results['MINORITY'].values
evaluation_df.drop(['ZIPCODE'], axis=1, inplace=True)

cols = evaluation_df.columns.tolist()
a, b = cols.index('MINORITY'), cols.index('RISK')
cols[b], cols[a] = cols[a], cols[b]

evaluation_df = evaluation_df[cols]
evaluation_df.head()

Write the evaluation sample dataframe to a CSV file in memory, then upload it to the project.

In [ ]:
evaluation_df.to_csv("policy_risk_openscale_eval.csv", index=False)
res = wslib.upload_file('policy_risk_openscale_eval.csv', overwrite=True)